In [1]:
import numpy as np 
import pandas as pd 
import plotly.express as px 
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 
import scipy as sp 


In [2]:
df = pd.read_excel("/Users/calebsiow/Documents/MATH 185/Project/Rice_MSC_Dataset.xlsx")

In [3]:
#subsample 7500 rows of data
df = df.sample(n=7500, random_state=1)


In [4]:
df.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,ALLdaub4L,ALLdaub4a,ALLdaub4b,ALLdaub4Y,ALLdaub4Cb,ALLdaub4Cr,ALLdaub4XX,ALLdaub4YY,ALLdaub4ZZ,CLASS
11591,12588,452.774,186.6662,87.4069,0.8836,126.5999,0.9676,13009,0.7696,2.1356,...,121.9372,63.3319,64.4904,111.9009,63.5161,63.3252,0.4200,0.4465,0.4772,Ipsala
52020,6625,299.812,114.3073,74.1271,0.7612,91.8434,0.9856,6722,0.7239,1.5420,...,110.4787,65.8663,58.3897,101.7653,68.8891,63.3700,0.3481,0.3555,0.4598,Karacadag
34666,7754,352.535,140.0796,72.0497,0.8576,99.3615,0.9666,8022,0.6156,1.9442,...,112.8153,64.1558,62.7876,103.4886,65.0696,63.6163,0.3520,0.3686,0.4174,Arborio
22169,6164,391.425,187.3339,42.6088,0.9738,88.5903,0.9742,6327,0.6989,4.3966,...,115.7075,63.2113,63.3481,105.9995,64.6054,62.7767,0.3693,0.3925,0.4352,Basmati
23049,9296,473.483,221.4163,54.3238,0.9694,108.7935,0.9711,9573,0.3742,4.0759,...,116.7207,65.1311,60.4665,107.4413,67.1169,63.5556,0.3891,0.4020,0.4878,Basmati


In [5]:
df.isnull().sum()
df.dropna(inplace=True)

In [6]:
y = df.CLASS 
X = df.drop(columns=['CLASS'])
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [7]:
#scipy svds 
from numpy.linalg import svd
from scipy.sparse.linalg import svds
Xtilde = X - np.mean(X, axis=0)
def compute_PCA(Xtilde):
    s = Xtilde.T @ Xtilde 
    U, S, V = np.linalg.svd(s)
    proj = Xtilde @ U
    return proj 
def compute_PCA_95(Xtilde):
    s = svd(Xtilde, compute_uv=False)
    frac_explained_scatter = np.cumsum(s)/np.sum(s)
    pca95 = np.argmax(frac_explained_scatter > 0.95) + 1
    U,S,_ = svds(Xtilde, k=pca95)
    proj = U @ (np.diag(S)).T
    return proj
proj = compute_PCA_95(Xtilde)

In [8]:
fig = px.scatter_3d(Xtilde, x=proj[:, 0], y=proj[:, 1], z=proj[:, 2], color=y,
                    template='plotly_dark', color_discrete_sequence=px.colors.qualitative.Pastel2, opacity = 0.7)
#label the axis PCA1 PCA2 PCA3
fig.update_layout(scene=dict(xaxis_title='PCA1',
                  yaxis_title='PCA2', zaxis_title='PCA3'))
#tighten the plot
fig.update_layout(width=1000, height=1000)
#make points smaller
fig.update_traces(marker=dict(
    size=3), selector=dict(mode='markers'))
fig.show()


In [9]:
#lda 
from scipy.linalg import eigh

Ytilde = proj 
def lda(X, labels):
    n,d = X.shape
    levels = np.unique(labels)
    c = len(levels)
    local_centroids = np.zeros((c+1,d))
    xtilde = np.zeros((n,d))
    group_counts = np.zeros((c,1))
    for i in range(c):
        inds_i = np.flatnonzero(labels == levels[i])
        group_counts[i] = np.sum(inds_i)
        local_centroids[i,:] = np.mean(X[inds_i,:],axis=0)
        xtilde[inds_i,:] = X[inds_i,:]-local_centroids[i,:]
    Sw = xtilde.T @ xtilde
    #sb = groupcounts * sum((local_centroids - global_centroids)@ (local_centroids - global_centroids).T)
    Sb = np.zeros((d,d))
    for i in range(c):
        Sb += group_counts[i] * np.outer(local_centroids[i,:]-np.mean(X,axis=0),local_centroids[i,:]-np.mean(X,axis=0))
    _, eigenvectors = np.linalg.eigh(np.linalg.pinv(Sw).dot(Sb))
    pc = X.dot(eigenvectors[:,::-1][:,:5])
    return pc, eigenvectors

Zlda, V = lda(Ytilde, y)    

    
fig = px.scatter_3d(Xtilde, x = Zlda[:,0], y = Zlda[:,1], z = Zlda[:,2], color = y, template = 'plotly_dark', color_discrete_sequence=px.colors.qualitative.Pastel2, opacity = 0.7)
#label the axis PCA1 PCA2 PCA3
fig.update_layout(scene = dict(xaxis_title = 'LDA1', yaxis_title = 'LDA2', zaxis_title = 'LDA3'))
#resize the plot
fig.update_layout(width=1000, height=1000)
#make points smaller
fig.update_traces(marker=dict(size=4), selector=dict(mode='markers'))
fig.show()



In [10]:
#kmeans clustering replicated 10 times
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5, random_state=1, n_init=10, max_iter=1000, tol=1e-6)
kmeans_labels = km.fit_predict(X)
#pca for plotting 
fig = px.scatter_3d(Xtilde, x=Zlda[:, 0], y=Zlda[:, 1], z=Zlda[:, 2], color=kmeans_labels,
                    template='plotly_dark', color_discrete_sequence=px.colors.qualitative.Pastel2, opacity=0.7)
#label the axis PCA1 PCA2 PCA3
fig.update_layout(scene = dict(xaxis_title = 'LDA1', yaxis_title = 'LDA2', zaxis_title = 'LDA3'))
#resize the plot
fig.update_layout(width=1000, height=1000)
#make points smaller
fig.update_traces(marker=dict(size=4), selector=dict(mode='markers'))
fig.show()

In [11]:
from scipy import linalg
from sklearn.neighbors import kneighbors_graph
from scipy import sparse


def generate_graph_laplacian(df, nn):
    """Generate graph Laplacian from data."""
    # Adjacency Matrix.
    connectivity = kneighbors_graph(X=df, n_neighbors=nn, mode='connectivity')
    adjacency_matrix_s = (1/2)*(connectivity + connectivity.T)
    # Graph Laplacian.
    graph_laplacian_s = sparse.csgraph.laplacian(
        csgraph=adjacency_matrix_s, normed=False)
    graph_laplacian = graph_laplacian_s.toarray()
    return graph_laplacian


graph_laplacian = generate_graph_laplacian(df=X, nn=8)



In [12]:
def affinity_matrix(X):
    from sklearn.neighbors import kneighbors_graph
    #pairwise dist 
    from scipy.spatial.distance import pdist, squareform
    #compute the pairwise distance
    dist = pdist(X, 'euclidean')
    #convert to squareform
    dist_sq = squareform(dist)
    #compute the affinity matrix
    A = -dist_sq
    A[np.diag_indices_from(A)] = 0
    A = np.exp(A)
    return A
W = affinity_matrix(proj)
W

array([[1.00000000e+00, 1.48338390e-11, 1.19241295e-08, ...,
        9.91769431e-12, 4.04840560e-10, 6.22870832e-06],
       [1.48338390e-11, 1.00000000e+00, 5.28914723e-07, ...,
        2.38068926e-09, 1.87208448e-05, 3.70769019e-10],
       [1.19241295e-08, 5.28914723e-07, 1.00000000e+00, ...,
        2.57509153e-06, 1.30995327e-05, 5.49257876e-07],
       ...,
       [9.91769431e-12, 2.38068926e-09, 2.57509153e-06, ...,
        1.00000000e+00, 5.85652842e-07, 4.98662541e-10],
       [4.04840560e-10, 1.87208448e-05, 1.30995327e-05, ...,
        5.85652842e-07, 1.00000000e+00, 6.67894869e-09],
       [6.22870832e-06, 3.70769019e-10, 5.49257876e-07, ...,
        4.98662541e-10, 6.67894869e-09, 1.00000000e+00]])

In [13]:
#convert diagonal to 0
W = W - np.diag(np.diag(W))
W

array([[0.00000000e+00, 1.48338390e-11, 1.19241295e-08, ...,
        9.91769431e-12, 4.04840560e-10, 6.22870832e-06],
       [1.48338390e-11, 0.00000000e+00, 5.28914723e-07, ...,
        2.38068926e-09, 1.87208448e-05, 3.70769019e-10],
       [1.19241295e-08, 5.28914723e-07, 0.00000000e+00, ...,
        2.57509153e-06, 1.30995327e-05, 5.49257876e-07],
       ...,
       [9.91769431e-12, 2.38068926e-09, 2.57509153e-06, ...,
        0.00000000e+00, 5.85652842e-07, 4.98662541e-10],
       [4.04840560e-10, 1.87208448e-05, 1.30995327e-05, ...,
        5.85652842e-07, 0.00000000e+00, 6.67894869e-09],
       [6.22870832e-06, 3.70769019e-10, 5.49257876e-07, ...,
        4.98662541e-10, 6.67894869e-09, 0.00000000e+00]])

In [14]:
#compute the Row Stochastic Matrix
def degree_matrix(W):
    D = np.diag(np.sum(W, axis=1))
    return D


D = degree_matrix(W)


def row_stochastic_matrix(D, W):
    #invD is the reciprocal of the degree matrix
    diag_reciprocal = np.reciprocal(np.diag(D))
    invD = np.diag(diag_reciprocal)
    #compute the row stochastic matrix
    S = invD.dot(W)
    return S 

P = row_stochastic_matrix(D, W)
P


array([[0.00000000e+00, 5.68444375e-12, 4.56942020e-09, ...,
        3.80053845e-12, 1.55138086e-10, 2.38689001e-06],
       [3.74787744e-11, 0.00000000e+00, 1.33634156e-06, ...,
        6.01498479e-09, 4.72995777e-05, 9.36774927e-10],
       [2.29921997e-09, 1.01985750e-07, 0.00000000e+00, ...,
        4.96531162e-07, 2.52586214e-06, 1.05908333e-07],
       ...,
       [1.27250322e-11, 3.05457565e-09, 3.30400613e-06, ...,
        0.00000000e+00, 7.51429825e-07, 6.39815738e-10],
       [4.12341814e-11, 1.90677216e-06, 1.33422528e-06, ...,
        5.96504350e-08, 0.00000000e+00, 6.80270232e-10],
       [3.59967210e-06, 2.14273462e-10, 3.17425083e-07, ...,
        2.88185213e-10, 3.85987336e-09, 0.00000000e+00]])

In [15]:
#eigs of P, return 5 largest magnitude eigenvalues and eigenvectors
from scipy.sparse.linalg import eigs
eigenvalues, eigenvectors = sp.sparse.linalg.eigsh(P, k=6, which='LM')
index = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[index]
eigenvectors = eigenvectors[:, index]



In [16]:
from sklearn.cluster import KMeans
#apply kmeans to the eigenvectors
km = KMeans(n_clusters=5, random_state=1, n_init=10, max_iter=1000, tol=1e-6)
spectral_labels = km.fit_predict(eigenvectors)
#pca for plotting
fig = px.scatter_3d(Xtilde, x=Zlda[:, 0], y=Zlda[:, 1], z=Zlda[:, 2], color=spectral_labels,
                    template='plotly_dark', color_discrete_sequence=px.colors.qualitative.Pastel2, opacity=0.7)
#label the axis PCA1 PCA2 PCA3
fig.update_layout(scene = dict(xaxis_title = 'PCA1', yaxis_title = 'PCA2', zaxis_title = 'PCA3'))
#resize the plot
fig.update_layout(width=1000, height=1000)
#make points smaller
fig.update_traces(marker=dict(size=4), selector=dict(mode='markers'))
fig.show()


